# 기계학습과 감정분석

## 학습 목표

- 기계학습, 인공신경망, 딥러닝의 개념을 이해한다
- 기계학습을 이용해 감정 분석을 실시한다
- 감정 분석의 계수들을 해석할 수 있다

## 기계학습

- Machine Learning
- 컴퓨터가 데이터로부터 패턴을 학습하게 하는 기법들
- 통계학과 밀접한 관련이 있는 인공지능 분야

### 기계학습의 종류

- 지도학습
- 비지도학습
- 강화학습

### 지도학습

- 입력변수(X)와 출력변수(Y)의 관계 패턴을 학습
- 학습 시에는 X, Y가 모두 있어야 함(지도)
- 활용 시에는 X만 있으면 Y를 예측

### 주의할 점

- Y가 없으면 지도학습은 불가능
  - Y가 없으면 만들어야함 -> 상당한 비용
  - 기계학습 주요 플레이어가 구글 등 거대기업인 이유
- 예: 블로그에 올라온 사용기가 광고인지 아닌지 알고 싶다
  - 광고 여부가 데이터에 없으면 학습시킬 수 없음

### 지도학습의 종류

- 회귀: 연속적인 예측(예: 가격, 만족도)
- 분류: 이산적인 예측(예: 강아지/고양이, 구매/미구매)

### 비지도학습

- 입력변수(X)를 잘 요약해 보여주는 변수 Z를 발견
- Z는 현실에 존재하는 어떤 대상이 아님 -> 정답이 없음

### 비지도학습의 용도

- 데이터가 너무 복잡해서 이해하기 쉬운 형태로 바꾸려고 할 때(예: 토픽 분석)
- 지도학습의 효율성을 높이기 위해

## 감정분석

- 감정 또는 감성(sentiment)
- 텍스트에 나타난 긍정/부정의 태도
- 기계학습 방식과 사전 방식

### 감정사전

- 긍정 표현, 부정 표현의 사전
- 해당 분야 전문가가 있으면 데이터 없이도 만들 수 있음
- 기계학습으로도 개발 가능

## IMDB 리뷰 데이터 분석

### 데이터 다운로드

In [1]:
import requests

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip'
res = requests.get(url)
with open('sentiment.zip', 'wb') as f:
    f.write(res.content)

### 압축 풀고 파일 열기

In [3]:
from zipfile import ZipFile
import pandas as pd

z = ZipFile('sentiment.zip')
f = z.open('sentiment labelled sentences/imdb_labelled.txt')
# csv 파일에서 칼럼 구분을 tab으로 
movie = pd.read_csv(f, sep='\t', header=None)

### 데이터 보기

In [4]:
movie.columns = ['review', 'sentiment']

In [5]:
movie.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


### IMDB 데이터 기계학습

- 긍정(1)/부정(0)을 예측 -> 분류

### TDM

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
#max_features: 최대 단어 수
#stop_words: 불용어(분석에 사용하지 않는 단어)
#다른 언어는 stop_words=[''] 형태로 
cv = CountVectorizer(max_features=1000, stop_words='english')

In [12]:
x = cv.fit_transform(movie['review'])

In [13]:
y = movie['sentiment']

In [23]:
freq = pd.DataFrame({'word': cv.get_feature_names(),
             'freq': x.sum(axis=0).flat})

freq.sort_values('freq').tail(10)

C:\Users\inyoung\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


,word,freq
550,movie,182
928,was,186
393,in,203
885,to,253
873,this,292
413,it,325
410,is,340
590,of,377
53,and,434
859,the,849


### 데이터 분할

- 학습용 데이터와 평가용 데이터로 분할
- 평가용 데이터는 10~20% 정도

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=1234)

- `test_size`: 평가용 데이터의 비율(`.2` = 20%)
- `random_state`: 난수를 생성해서 무작위로 나눌 때, 난수의 초기값을 설정. 
  - 초기값이 같으면 항상 같은 방식으로 나눔
  - 재현을 위해 사용

### 로지스틱 회귀분석

- 대표적인 분류 모형
- 이름은 회귀이나 실제로는 분류
- 로지스틱 회귀분석을 확장한 것이 인공신경망(=딥러닝)

In [20]:
from sklearn.linear_model import LogisticRegressionCV

In [24]:
model = LogisticRegressionCV()

In [25]:
model.fit(x_train, y_train)

#Cs = 10 => 10개 설정값을 시도하고 좋은 값. 

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

### 성능 평가

In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [27]:
y_pred = model.predict(x_test)

In [29]:
# 실제 분석시에는 y_test가 없음. 여기는 y가 있으니까 성능 평가 가능. 
accuracy_score(y_test, y_pred)

0.7533333333333333

### 혼돈 행렬

In [103]:
%%html
<table style="font-size:2em"><tr><td></td><td></td><th colspan=2 style="text-align:center">예측</th></tr><tr><td></td><td></td><th>부정</th><th>긍정</th></tr><tr><th rowspan=2>실제</td><th>부정</th><td>TN</td><td>FP</td></tr><tr><th>긍정</th><td>FN</td><td>TP</td></tr></table>

In [31]:
confusion_matrix(y_test, y_pred)

array([[52, 21],
       [16, 61]], dtype=int64)

$$
\text{accuracy} = \frac{T}{T + F}
$$

### 계수 분석

In [33]:
word_coef = pd.DataFrame({
    'word': cv.get_feature_names(),
    'coef': model.coef_.flat
})

C:\Users\inyoung\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [34]:
word_coef.sort_values('coef').head()

,word,coef
91,bad,-1.395947
984,would,-0.780205
89,awful,-0.720917
230,even,-0.719824
571,no,-0.685359


In [35]:
word_coef.sort_values('coef').tail()

,word,coef
972,wonderful,0.600632
764,see,0.601679
467,liked,0.649155
295,funny,0.667721
326,great,0.760885


## 네이버 영화평

- 네이버 영화에서 분석하고 싶은 영화 페이지로 들어감
- 네티즌 별점 클릭
- 페이지 번호 우클릭 후 주소 복사

In [58]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150689&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}'

### 리뷰 수집

In [69]:
import requests
import lxml.html

# 리뷰와 별점을 모을 빈 리스트를 만든다
reviews = []
scores = []

for page in range(1, 30):  # 1~29페이지까지 반복
    res = requests.get(url.format(page))  # 각 페이지에 접속한다
    root = lxml.html.fromstring(res.text)  # html을 처리한다

    # 리뷰를 가져와 reviews에 추가한다
    for review in root.cssselect('.score_reple p'):
        reviews.append(review.text_content())

    # 별점을 가져와 scores에 추가한다
    for score in root.cssselect('.score_result .star_score em'):
        scores.append(int(score.text_content()))

### 표 만들기

In [70]:
import pandas as pd

df = pd.DataFrame({
    'score': scores, 
    'review': reviews
})

### 긍/부정 표시

In [74]:
import numpy as np

In [76]:
df['sentiment'] = np.where(df['score'] > 5, 1, 0)

In [77]:
df.head()

,score,review,sentiment
0,6,영화는 괜찮음 생각보다. 근데 얘네는 왜 핵얘기 나올때마다 지들이 진주만 공격한건 ...,1
1,10,관람객이시하라 사토미 예쁘다,1
2,1,이래서 일본은 애니를 보나보다.,0
3,1,야 진짜 무슨 구연동화냐 겁나 허접한게 느껴짐,0
4,3,호불호 소리는 많이 들었지만 난 불호다. 고질라가 때려부수는거 보러갔더니 일본애들이...,0


### 저장

In [78]:
df.to_csv('movie_review.csv', encoding='utf8', index=False)

### 불러오기

In [79]:
df = pandas.read_csv('movie_review.csv', encoding='utf8')

### WPM 학습

In [83]:
from subword_nmt.learn_bpe import learn_bpe
import io

with open('영화평BPE.txt', 'w', encoding='utf8') as outfile:
    infile = io.StringIO(' '.join(df['review']))
    learn_bpe(infile, outfile, 1000)

In [84]:
from subword_nmt.apply_bpe import BPE

with open('영화평BPE.txt', encoding='utf8') as f:
    bpe = BPE(f, separator='~')

### TDM 만들기

In [85]:
def tokenizer_wpm(text):
    tokens = bpe.process_line(text)
    tokens = tokens.split()
    return [t for t in tokens
            if (not t.endswith('~') and len(t) > 1) or len(t) > 2]

In [86]:
cv_wpm = CountVectorizer(max_features=1000, tokenizer=tokenizer_wpm)

In [87]:
tdm = cv_wpm.fit_transform(df['review'])

### 토큰 빈도

In [88]:
freq = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'n': tdm.sum(axis=0).flat
})

C:\Users\user\Anaconda3\envs\py36\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [91]:
freq.sort_values('n').tail(10)

,word,n
580,일본~,23
76,관람객~,24
491,으로,26
3,..,31
840,하는,37
436,영화,37
155,다.,39
439,영화~,40
579,일본,40
65,고질라,51


### 데이터 분할

In [92]:
x_train, x_test, y_train, y_test = train_test_split(
    tdm, df['sentiment'], test_size=.2, random_state=1234)

### 학습

In [93]:
model = LogisticRegressionCV()

In [94]:
model.fit(x_train, y_train)

C:\Users\user\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

### 성능 평가

In [95]:
y_pred = model.predict(x_test)

In [96]:
accuracy_score(y_test, y_pred)

0.7241379310344828

### 계수 분석

In [98]:
word_coef = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'coef': model.coef_.flat
})

C:\Users\user\Anaconda3\envs\py36\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [102]:
word_coef.sort_values('coef').head(10)

,word,coef
98,그냥,-0.136674
830,하고,-0.133008
7,..~,-0.124938
158,다가,-0.101295
236,만들~,-0.093658
256,무슨,-0.090435
314,수준~,-0.086677
191,들의,-0.086003
168,대사~,-0.085291
338,아깝다,-0.083891


In [101]:
word_coef.sort_values('coef').tail(10)

,word,coef
278,보다는,0.097074
697,정치,0.098877
448,영화를,0.100680
768,최고~,0.101680
814,풍자~,0.103882
86,괴수~,0.104106
491,으로,0.114102
155,다.,0.122833
282,보여~,0.139746
76,관람객~,0.347984
